# Download and Clean Dataset


In [1]:
import pandas as pd
import numpy as np

## Import data from source and explore it

In [2]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cement cubic meter, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. the first concrete sample is much closer to the second concrete sample

In [3]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

### Split data into predictors and target

In [5]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Check of the predictors and the target dataframes.

In [6]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Normalize the data by substracting the mean and dividing by the standard deviation.

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [9]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## Import Keras

In [10]:
import keras

Using TensorFlow backend.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [11]:
from keras.models import Sequential
from keras.layers import Dense

Build a Neural Network that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

In [12]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

 Randomly split the data into a training and test sets

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)


# Train and Test the Network

In [14]:
# call the model
model = regression_model()
# fit the model with 50 epochs
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 123us/step - loss: 1581.7399
Epoch 2/50
721/721 [==============================] - 0s 28us/step - loss: 1561.9508
Epoch 3/50
721/721 [==============================] - 0s 25us/step - loss: 1542.2231
Epoch 4/50
721/721 [==============================] - 0s 32us/step - loss: 1522.0133
Epoch 5/50
721/721 [==============================] - 0s 33us/step - loss: 1501.3241
Epoch 6/50
721/721 [==============================] - 0s 28us/step - loss: 1479.5236
Epoch 7/50
721/721 [==============================] - 0s 33us/step - loss: 1456.7348
Epoch 8/50
721/721 [==============================] - 0s 31us/step - loss: 1433.0434
Epoch 9/50
721/721 [==============================] - 0s 37us/step - loss: 1408.1269
Epoch 10/50
721/721 [==============================] - 0s 34us/step - loss: 1382.2685
Epoch 11/50
721/721 [==============================] - 0s 35us/step - loss: 1355.2508
Epoch 12/50
721/721 [==============================] - 0s 30us

## Evaluating the model on the test data

In [15]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 54us/step


304.6997691527925

##  Compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [16]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

304.69977397841853 0.0


### Create a list of 50 mean squared errors.

In [17]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("The mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 165.26746701731264
MSE 2: 144.2230524353225
MSE 3: 90.53341027911041
MSE 4: 75.8494011962298
MSE 5: 60.71592454385603
MSE 6: 58.25827860909372
MSE 7: 55.33983908347713
MSE 8: 40.479712501698714
MSE 9: 41.1901425605453
MSE 10: 43.22617049973373
MSE 11: 39.411016421024854
MSE 12: 38.71380401660709
MSE 13: 45.45026439061829
MSE 14: 41.058586589726815
MSE 15: 38.98953050780065
MSE 16: 35.56267121694621
MSE 17: 35.57736328890409
MSE 18: 33.96259810932246
MSE 19: 34.34621107231066
MSE 20: 35.73389777396489
MSE 21: 33.60293323553881
MSE 22: 32.18467442040305
MSE 23: 33.22052099480984
MSE 24: 35.17321067489081
MSE 25: 35.67760900849277
MSE 26: 38.63200766060345
MSE 27: 35.60078536345349
MSE 28: 31.71029667406792
MSE 29: 36.99054025446327
MSE 30: 34.418120307058196
MSE 31: 33.475951407719585
MSE 32: 29.340408417785053
MSE 33: 33.101309878154865
MSE 34: 35.22128045288876
MSE 35: 33.607877453554025
MSE 36: 38.59621409073617
MSE 37: 35.08244495947384
MSE 38: 35.27805572806053
MSE 39: 34.481